
## Gen AI Internship HW3 Hayk Nalchajyan

### Project1
In this programming exercise, you are provided with a boilerplate code for defining two classes, Point2D and Point3D, representing points in a 2-dimensional and 3-dimensional space, respectively. The provided code uses special methods like __init__, __eq__, __hash__, and properties to encapsulate the data and provide certain functionalities. However, the code is not generic, and it involves some repetition.
Your task is to refactor the provided code to make it more generic and DRY (Don’t Repeat Yourself) by using metaclasses. Metaclasses allow you to create classes dynamically, enabling you to define common behaviors for multiple classes in an elegant and efficient manner. Your refactored code should be able to generate N-dimensional point classes with the same functionalities while minimizing duplication of code.

### Hints
Analyze the provided Point2D and Point3D classes and identify the common functionalities that can be generalized for N-dimensional points.

Create a custom metaclass called SlottedStruct that defines these common functionalities, such as initializing the point’s coordinates, comparing points for equality, computing the hash, and generating string representations.

Modify the Point2D and Point3D classes to use the SlottedStruct metaclass instead of the current implementations. Make sure the classes still function correctly after the changes.

Test your refactored code with various N-dimensional points (e.g., Point4D, Point5D) to verify that the generic implementation works as expected.

Submit your refactored code along with a brief explanation of how you used metaclasses to achieve a more generic and DRY implementation.

In [1]:
class SlottedStructMeta(type):
    def __new__(cls, name, bases, dct):
        if 'num_coords' not in dct:
            raise ValueError("The 'num_coords' attribute must be provided in the class definition.")

        if not isinstance(dct['num_coords'], int) or dct['num_coords'] <= 0:
            raise ValueError("The 'num_coords' attribute must be a positive integer representing the number of dimensions.")

        dct['__slots__'] = ['_coordinates']

        return super().__new__(cls, name, bases, dct)

    def __init__(cls, name, bases, dct):
        super().__init__(name, bases, dct)

        for i in range(cls.num_coords):
            def get_coord(self, index=i):
                return self._coordinates[index]

            def set_coord(self, value, index=i):
                self._coordinates[index] = value

            setattr(cls, f'coord{i + 1}', property(get_coord, set_coord))

    def __eq__(cls, other):
        return cls.num_coords == other.num_coords and cls.__dict__ == other.__dict__

    def __hash__(cls):
        return hash(cls.num_coords) ^ hash(tuple(sorted(cls.__dict__.items())))

    def __repr__(cls):
        coords_str = ', '.join(str(getattr(cls, f'coord{i + 1}')) for i in range(cls.num_coords))
        return f"{cls.__name__}({coords_str})"


class Point2D(metaclass=SlottedStructMeta):
    num_coords = 2

    def __init__(self, x, y):
        self._coordinates = [x, y]


class Point3D(metaclass=SlottedStructMeta):
    num_coords = 3

    def __init__(self, x, y, z):
        self._coordinates = [x, y, z]


class Point4D(metaclass=SlottedStructMeta):
    num_coords = 4

    def __init__(self, x, y, z, w):
        self._coordinates = [x, y, z, w]


if __name__ == "__main__":
    p1 = Point2D(1, 2)
    p2 = Point2D(1, 2)
    p3 = Point2D(2, 3)

    print(p1 == p2)
    print(p1 == p3)

    p4 = Point3D(1, 2, 3)
    p5 = Point3D(1, 2, 3)
    p6 = Point3D(2, 3, 4)

    print(p4 == p5)
    print(p4 == p6)

    p7 = Point4D(1, 2, 3, 4)
    p8 = Point4D(1, 2, 3, 4)
    p9 = Point4D(2, 3, 4, 5)

    print(p7 == p8)
    print(p7 == p9)


False
False
False
False
False
False


### Project2
There’s another pattern we can implement using metaprogramming - Singletons.
If you read online, you’ll see that singleton objects are controversial in Python. I’m not going to get into a debate on this, other than to say I do not use singleton objects, not because I have deep thoughts about it (or even shallow ones for that matter), but rather because I have never had a need for them.
However, the question often comes up, so here it is - the metaclass way of implementing the singleton pattern. Whether you think you should use it or not, is entirely up to you! We have seen singleton objects - objects such as `None`, `True` or `False` for example. No matter where we create them in our code, they always refer to the **same** object. We can recover the type used to create `None` objects:

NoneType = type(None)

And now we can create multiple instances of that type:

n1 = NoneType() 

n2 = NoneType()

The same holds true for booleans:

b1 = bool([]) 

b2 = bool("")

There is no built-in mechanism to Python for singleton objects, so we have to do it ourselves.
The basic idea is this:
When an instance of the class is being created (but **before** the instance is actually created), check if an instance has already been created, in which case return that instance, otherwise, create a new instance and store that instance reference somewhere so we can recover it the next time an instance is requested.
We could do it entirely in the class itself, without any metaclasses, using the `__new__` method. We can start with this:

In [2]:
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]


class SingletonExample(metaclass=SingletonMeta):
    def __init__(self):
        self.name = 'singleton_example'
        self.value = 42


instance1 = SingletonExample()
instance2 = SingletonExample()

print(instance1 is instance2)
print(vars(instance1))
print(vars(instance2))


True
{'name': 'singleton_example', 'value': 42}
{'name': 'singleton_example', 'value': 42}
